### **Setting**

Assume $P_{X|0} ~conforms~ N(0,1)$, $P_I ~conforms~ N(0.5,1)$, $P_{X|1} ~conforms~ N(1,1)$

First let $ϵ$ be 0

Assume $C_S - C_I = N(0,0.25)$

#### **1. Get the cdf, pdf, and visualize the distributions**

In [1]:
import scipy.stats
from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import uniform
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Visualize the pdf
mu_0 = 0
mu_1 = 0.5
mu_2 = 1
sd = 1
x = np.linspace(-4,4,1000)
plt.figure(figsize = (4,3))
plt.plot(x, norm.pdf(x, mu_0, sd),'r-', linestyle = 'solid', label='unqualified')
plt.plot(x, norm.pdf(x, mu_1, sd),'g-', linestyle = 'dashed',label='fail to improve')
plt.plot(x, norm.pdf(x, mu_2, sd),'b-', linestyle = 'dotted', label='qualified')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('x',size=14)
plt.ylabel('pdf',size=14)
plt.legend()
plt.savefig('gaussian.pdf',bbox_inches='tight')

In [ ]:
# Visualize the cdf
mu_0 = 0
mu_1 = 0.5
mu_2 = 1
sd = 1
x = np.linspace(-4,4,1000)
plt.figure(figsize = (4,3))
plt.plot(x, norm.cdf(x, mu_0, sd),'r-',linestyle = 'solid' ,label='unqualified')
plt.plot(x, norm.cdf(x, mu_1, sd),'g-', linestyle = 'dashed',label='fail to improve')
plt.plot(x, norm.cdf(x, mu_2, sd),'b-', linestyle = 'dotted',label='qualified')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('x',size=14)
plt.ylabel('cdf',size=14)
plt.legend()
plt.savefig('gaussian_cdf.pdf',bbox_inches='tight')

In [ ]:
# Visualize Ci-Cs
mu_c = 0
std_c = 0.25
x = np.linspace(-1,1,1000)
plt.figure(figsize = (8,5))
plt.plot(x, norm.pdf(x, mu_c, std_c),'r-', lw=3, alpha=0.6, label='cost-pdf')
plt.plot(x, norm.cdf(x, mu_c, std_c),'b-', lw=3, alpha=0.6, label='cost-cdf')
plt.legend()

#### 2. Optimal Decision Maker vs Strategic Optimal Decision Maker

For non-strategic decision maker, we assume $u^+ = u^- = 1$

In [5]:
eps = 0.2
u = 1
q = 0.5
alpha = 0.6

# Optimal Threshold without strategy
def get_opt_theta_no_strategic():
    """
    get optimal threshold under non-strategic situation
    """
    val = u*(1-alpha)/(u*alpha)
    from scipy.optimize import fsolve
    func = lambda theta: norm.pdf(theta,mu_2,sd)/norm.pdf(theta,mu_0,sd) - val
    root = fsolve(func,0.2)
    return root[0]

opt_no = get_opt_theta_no_strategic()

def get_nonstrat_utility(theta):
    return u*alpha*(1-norm.cdf(theta,mu_2,sd))-u*(1-alpha)*(1-norm.cdf(theta,mu_0,sd))


In [ ]:
U_no = get_nonstrat_utility(opt_no)
print('non-strategic optimal threshold is', U_no)
print('non-strategic utility is', opt_no)

### Strategic Utility

We decompose strategic utility into two parts:

Non-strategic utility and $\Phi(\theta)$

In [7]:
# Get manipulation Prob
def get_manip_prob(theta, q, eps):
    """
    function to get manip prob under q and eps
    """
    value = (1-q)*(norm.cdf(theta, mu_1, sd) - norm.cdf(theta, mu_2, sd)) - eps*(1 - norm.cdf(theta, mu_2, sd))
    return norm.cdf(value, mu_c, std_c)

In [8]:
# Get Phi, which is strategic - non-strategic
def get_Phi(theta, q, eps, k1 = 1, k2 = 1, k3 = 1):
    """
    Get Phi, which is strategic - non-strategic
    k1, k2, k3 measure our preferences for 3 kinds of behaviors
    larger k1 means we prefer improvement
    larger k2 means we do not prefer failure of improvement
    larger k3 means we do not prefer manipulation
    k1=k2=k3=1 is the naive difference
    """
    # manipulation probability
    pm = get_manip_prob(theta, q, eps)
    # Reverse Benefit
    b1 = k1*(1-pm)*q*(2 - norm.cdf(theta, mu_0, sd) - norm.cdf(theta, mu_2, sd))
    # Loss of Failure of Improvement 
    l1 = k2*(1-pm)*(1-q)*(norm.cdf(theta, mu_0, sd) - norm.cdf(theta, mu_1, sd))
    # Loss of manipulation
    l2 = k3*pm*((1-eps)*(1 - norm.cdf(theta,mu_2,sd))-(1-norm.cdf(theta,mu_0,sd)))
    return u*(1-alpha)*(b1-l1-l2)

In [9]:
def get_strat_optimal(q, eps, k1=1, k2=1, k3=1):
    def get_strat_utility(theta, k10 = k1, k20 = k2, k30 = k3, q0 = q, eps0 = eps):
        phi = get_Phi(theta,q0,eps0,k10,k20,k30)
        U = get_nonstrat_utility(theta) + phi
        return -U

    from scipy.optimize import minimize_scalar
    res = minimize_scalar(get_strat_utility)
    return res.x, -get_strat_utility(res.x)


In [ ]:
get_strat_optimal(0.5,0.2)

In [28]:
# for fixed q, vary eps
def vary_eps(q_list = [0,0.3,0.6,1], eps_list = np.linspace(0,1,100)):
  for i in range(len(q_list)):
      q = q_list[i]
      x = np.zeros(100)
      u = np.zeros(100)
      m = np.zeros(100)
      j = 0
      for eps in eps_list:
        x1,u1 = get_strat_optimal(q,eps)
        x[j] = max(x1,-5)
        u[j] = u1
        m[j] = get_manip_prob(x1,q,eps)
        j += 1
      df = pd.DataFrame({'eps': np.linspace(0,1,100), 'theta_max': x, 'max utility': u, 'manipulation prob': m})
      axes[int(i/2)][int(i%2)].tick_params(axis='both', which='major', labelsize=14)
      axes[int(i/2)][int(i%2)].plot(df['eps'], df['theta_max'],'r-', label=r'$\theta^*$')
      axes[int(i/2)][int(i%2)].plot(np.linspace(0,1,100), [opt_no]*100, 'b-', label = r'$\widehat{\theta}^{*}$')
      axes[int(i/2)][int(i%2)].set_xlabel(r'$\epsilon$',fontsize = 14)
      axes[int(i/2)][int(i%2)].set_ylabel(r'$\theta$',fontsize=14)
      axes[int(i/2)][int(i%2)].legend(fontsize = 11)
      axes[int(i/2)][int(i%2)].set_title('q = %.2f'%q, fontsize=14)


In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(8, 6)
fig.tight_layout(pad=3.0)
vary_eps()
plt.tight_layout()
fig.savefig('threshold.pdf')

### Illustration of different parameter combinations

In [30]:
def strat_u(theta, q, eps):
    value = (1-q)*(norm.cdf(theta, mu_1, sd) - norm.cdf(theta, mu_2, sd)) - eps*(1 - norm.cdf(theta, mu_2, sd))
    pm = norm.cdf(value, mu_c, std_c)
    U = u_p*(alpha + (1-alpha)*(1-pm)*q)*(1-norm.cdf(theta, mu_2, sd))-u_n*(1-alpha)*((1-eps)*pm*(1-norm.cdf(theta, mu_2, sd)) + (1-pm)*(1-q)*(1-norm.cdf(theta, mu_1, sd)))
    return U

def non_strat_u(theta):
    return u_p*alpha*(1-norm.cdf(theta,mu_2,sd))-u_n*(1-alpha)*(1-norm.cdf(theta,mu_0,sd))

In [ ]:
u_p = u_n = 1
alpha = 0.6
eps = 0
q = 0.5
theta = np.linspace(-5,5,1000)
func1 = np.vectorize(strat_u, excluded = ['q','eps'])
value = func1(theta,q,eps)
plt.figure(figsize = (4,3))
plt.plot(theta, value,color = 'blue')
plt.xlabel(r'$\theta$', fontsize = 14)
plt.ylabel('Strategic utility', fontsize = 14)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.tight_layout()
plt.savefig('0050.pdf')

In [ ]:
u_p = u_n = 1
alpha = 0.6
eps = 0.75
q = 0.25
theta = np.linspace(-5,5,1000)
func1 = np.vectorize(strat_u, excluded = ['q','eps'])
value = func1(theta,q,eps)
plt.figure(figsize = (4,3))
plt.plot(theta, value, color = 'blue')
plt.xlabel(r'$\theta$', fontsize = 14)
plt.ylabel('Strategic utility', fontsize = 14)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.tight_layout()
plt.savefig('7525.pdf')

In [ ]:
u_p = u_n = 1
alpha = 0.3
eps = 0
q = 0.5
theta = np.linspace(-5,5,1000)
func1 = np.vectorize(strat_u, excluded = ['q','eps'])
value = func1(theta,q,eps)
plt.figure(figsize = (4,3))
plt.plot(theta, value,color = 'blue')
plt.xlabel(r'$\theta$', fontsize = 14)
plt.ylabel('Strategic utility', fontsize = 14)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.tight_layout()
plt.savefig('0050n.pdf')

In [ ]:
u_p = u_n = 1
alpha = 0.3
eps = 0.75
q = 0.25
theta = np.linspace(-5,5,1000)
func1 = np.vectorize(strat_u, excluded = ['q','eps'])
value = func1(theta,q,eps)
plt.figure(figsize = (4,3))
plt.plot(theta, value,color = 'blue')
plt.xlabel(r'$\theta$', fontsize = 14)
plt.ylabel('Strategic utility', fontsize = 14)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.tight_layout()
plt.savefig('7525n.pdf')

### Different Policies

Visualize the influences of different $k_1,k_2$


#### k1

In [20]:
# for fixed q, vary eps

def vary_eps_k1(q, eps_list = np.linspace(0,1,100), k_list = [100], filename = 'k1.pdf'):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(4,3)
    ax1.plot(eps_list, [opt_no]*100, 'b-', label = r'$\widehat{\theta}^*$')
    for k in k_list:
        x = np.zeros(100)
        u = np.zeros(100)
        us = np.zeros(100)
        m = np.zeros(100)
        i = 0  
        for eps in eps_list:
            x1,u1 = get_strat_optimal(q,eps, k1 = k, k2 = 1, k3 = 1)
            if abs(x1) > 10:
                if x1 != abs(x1):
                    x1 = -10
                else:
                    x1 = 10
            x[i] = x1
            phi = get_Phi(x1,q,eps,1,1,1)
            U = get_nonstrat_utility(x1) + phi
            u[i] = U
            us[i] = u1
            m[i] = get_manip_prob(x1,q,eps)
            i += 1
    
        df = pd.DataFrame({'eps': np.linspace(0,1,100), 'theta_max': x, 'max utility': u, 'manipulation prob': m})
        ax1.plot(df['eps'], df['theta_max'], label=r'$\theta^*(k_1 = %.2f)$'%k, color = 'red')

    ax1.legend(fontsize=12)
    ax1.tick_params(axis='both', which='major', labelsize=14)
    ax1.set_xlabel(r'$\epsilon$',fontsize = 14)
    ax1.set_ylabel(r'$\theta$',fontsize = 14)
    ax1.set_xticks(np.linspace(0,1,6))
    ax1.set_yticks(np.linspace(-9,3,4))
    plt.tight_layout()
    fig.savefig(filename)

In [ ]:
alpha = 0.6
opt_no = get_opt_theta_no_strategic()
U_no = get_nonstrat_utility(opt_no)
vary_eps_k1(q = 0.01, k_list = [10], filename = 'k1.pdf')
vary_eps_k1(q = 0.99, k_list = [10], filename = 'k1second.pdf')

In [ ]:
alpha = 0.3
opt_no = get_opt_theta_no_strategic()
U_no = get_nonstrat_utility(opt_no)
vary_eps_k1(q = 0.01, k_list = [100], filename = 'k1031.pdf')
vary_eps_k1(q = 0.01, k_list = [100], filename = 'k1032.pdf')

#### k2

In [21]:
# for fixed q, vary eps
def vary_eps_k2(q, eps_list = np.linspace(0,1,100), k_list = [100], filename = 'k2.pdf'):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(4,3)
    ax1.plot(eps_list, [opt_no]*100, 'b-', label = r'$\widehat{\theta}^*$')
    for k in k_list:
        x = np.zeros(100)
        u = np.zeros(100)
        us = np.zeros(100)
        m = np.zeros(100)
        i = 0  
        for eps in eps_list:
            x1,u1 = get_strat_optimal(q,eps, k1 = 1, k2 = k, k3 = 1)
            # if abs(x1) > 10:
            #     if x1 != abs(x1):
            #         x1 = -10
            #     else:
            #         x1 = 10
            if x1 > 5:
                x1 = 5
            elif x1 < -5:
                x1 = -5
            x[i] = x1
            phi = get_Phi(x1,q,eps,1,1,1)
            U = get_nonstrat_utility(x1) + phi
            u[i] = U
            us[i] = u1
            m[i] = get_manip_prob(x1,q,eps)
            i += 1
    
        df = pd.DataFrame({'eps': np.linspace(0,1,100), 'theta_max': x, 'max utility': u, 'manipulation prob': m})
        ax1.plot(df['eps'], df['theta_max'], lw=2, alpha=0.6, label=r'$\theta^*(k_2 = %.2f)$'%k, color = 'red')

    ax1.legend(fontsize=12)
    ax1.tick_params(axis='both', which='major', labelsize=14)
    ax1.set_xticks(np.linspace(0,1,6))
    ax1.set_yticks(np.linspace(-5,5,6))
    ax1.set_xlabel(r'$\epsilon$',fontsize = 14)
    ax1.set_ylabel(r'$\theta$',fontsize = 14)
    plt.tight_layout()
    fig.savefig(filename)

In [ ]:
alpha = 0.3
opt_no = get_opt_theta_no_strategic()
U_no = get_nonstrat_utility(opt_no)
vary_eps_k2(q = 0.01, k_list = [100], filename = 'k2031.pdf')
vary_eps_k2(q = 0.99, k_list = [100], filename = 'k2039.pdf')

In [ ]:
alpha = 0.6
opt_no = get_opt_theta_no_strategic()
U_no = get_nonstrat_utility(opt_no)
vary_eps_k2(q = 0.01, k_list = [100], filename = 'k2061.pdf')
vary_eps_k2(q = 0.99, k_list = [100], filename = 'k2069.pdf')